# Theory

## Preprocessing

### Dataset

The input dataset (taken from
[this](https://www.kaggle.com/qks1lver/amex-nyse-nasdaq-stock-histories)
Kaggle challenge) consists of approximately $8000$ CSV files, each of which
contains historical price records for a stock symbol from 1970
to 2018. Each record consists of the following columns:

The high, low, open, and close prices are historical prices as
recorded by the market on that day. The `adjclose` column provides an
adjusted close price that accounts for events like splits, which
retroactively affect historical price data. For example, if a company doubles its number of existing shares, the
price per share will be reduced by half. The adjusted close price
accounts for these changes, providing a shared scale on which to
measure closing price over the history of the stock.

### Filtering

Several filtering techniques can be applied to the input dataset in
order to produce higher quality training examples. One such technique
is the removal records older than a parameterized year.
Since the dataset contains records dating back to 1970, the
ability to constrain to more recent records may prove useful in
capturing market behavior under modern trading styles.

Another filtering technique is to remove penny stocks, which tend to exhibit greater
volatility due to their low price per share. This is accomplished
through the use of a parameterized closing price threshold. If the
average closing price of a stock over the constrained date window is
below the parameterized threshold, that stock will be excluded
entirely from the training example set.

### Positional Encoding

A positional encoding strategy was used to pass date information from
each record to the neural network in a numerically stable form. The underlying assumption
is that stocks may behave differently at different times of year (e.g.
during quarterly earnings reports, end of year, etc.), and thus this
information could prove useful to the network. The following function
was used to calculate a positional encoding based on the day of year:

$$
f(x) = \sin\left(\frac{\pi x}{365}\right)
$$

This function transforms a day of year on the interval $[1, 365]$ into a real number on a
continuous interval from $(0, 1]$. Under this transformation, day
$365$ and day $1$ will have similar values, capturing the idea that
December 31st and January 1st are close to each other. It is worth
noting that each point under this transformation will be produced by
two points in the original "day of year" space. This is not ideal, but
the network should be able to compensate based on whether or not the
point lies on an increasing or decreasing region in the encoded space.

### Price Adjustment

The close and adjusted close price given in the original dataset were used to
rescale the other original price metrics (high, low, open). This was
accomplished by apply the following transformation to each unscaled
price in the record:

$$
\text{scaled} = \text{unscaled} \cdot
\frac{\text{adjclose}}{\text{close}}
$$

After this point the unscaled prices were discarded. This places all
prices on a shared scale that is invariant to stock splits and other
price-altering events, improving continuity.

### Future Price Calculation

A future percent change in closing price was calculated for each
record by looking $x$ days into the future and calculating a pointwise
or aggregate measure of closing price over those $x$ days. One such
strategy is to calculate a pointwise percent change in
price as follows, where $i$ indicates an index of the record to be
labeled:

$$
\Delta = \frac{P_{i+x} - P_{i}}{P_i} * 100
$$

Another strategy is to calculate an aggregate of some price metric
over $x$ days in the future and calculate percent change from the present
price to that aggregate price. For example, one could calculate an average closing
price over the following $x$ days and then calculate percent change
in closing price from the present to that average.

$$
\begin{aligned}
	&\Delta = \frac{a - P_{i}}{P_i} * 100
	&\text{where}&
	&a = \frac{1}{x}\sum_{j=i+1}^{i+x} P_j
\end{aligned}
$$

The decision to use a pointwise or aggregate basis for percent change
calculation is dependent on the choice of window size among other
things. The choice of window size is primarily determined by the style
of trading to be used, along with the interval between records (if
this approach were to be applied to by the minute data for day
trading). For experimentation, percent change in future price was
calculated relative to the average closing price over a future $3$ day
window.

### Spark ML Pipeline

A parameterized Spark ML pipeline is used to extract feature vectors and produce
discretized labels for each record. The pipeline consists of the
following stages:

1. `VectorAssembler` - Collects feature columns into a single vector
	for processing

2. Normalizer - A strategy to normalize each feature to a shared
	domain. Can be one of the following, or excluded entirely:
	* `StandardScaler` - Rescales each feature to zero mean unit variance
	* `MaxAbsScaler` - Rescales each feature to the range $[0, 1]$,
		**preserving sparsity**

3. Labeler - A strategy to assign examples a discretized label based
	 on future percent change. Can be one of the following, or excluded entirely:
	* `Bucketizer` - Label based on fixed sized buckets of percent
		change
	* `QuantileDiscretizer` - Label based on variable sized buckets such
		that each each class contains an equal number of examples

The each record in the output of this pipeline will have a vector of
(normalized) features for that day, a future percent change value, and
an integer label if requested. By retaining both the future percent
change and a discretized label, each record is suitable for use with
classification or regression networks.

It is important to consider the distribution of examples among the
possible classes when using `Bucketizer` as a labeling strategy.
It was observed that percent change tends to concentrate near zero,
meaning that the bucket that overlaps with zero will have a
potentially large number of examples. This is likely to become more of
an issue with larger future window sizes, as price will have more time
to regress to the mean.

Conversely, when using
`QuantileDiscretizer` a uniform distribution of examples over each
class will be created by compressing or expanding bucket ranges such
that the $n$'th bucket contains the top $1/n$'th percent change values.
This will manifest as buckets that span only a few percentage points of
percent change, taxing the network's ability to distinguish these
classes. However, it does
eliminate concerns that the network will exhibit a bias towards zero
price movement predictions.

During experimentation the `MaxAbsScaler` was used as a normalization
strategy and the `Bucketizer` was used for a labeling strategy with
a bucket parameter list of $-5,-2,2,5$. This produced five discretized
classes in the resultant training set. Unequal bucket widths
were important in creating a somewhat balanced distribution of
examples among the labels while not forcing the network to learn
unreasonably small differences between the classes.

## Creating Training Examples

Having extracted numerically stable features and labels from the
original dataset, steps must now be taken to construct complete training
examples. Taking inspiration from human styles of trading, the
decision was made to create training examples by aggregating
chronologically ordered feature vectors over a sliding historical window into a
feature matrix.

As a concrete example, consider training example $i$ with features
$h,l,o,c,v,p$ and a window size of $x$. Example $i$ would then consist
of features over the $x$ previous days for that stock as follows:

$$
\text{example}_i = \begin{bmatrix}
	h_i & l_i & o_i & c_i & v_i & p_i \\
	h_{i-1} & l_{i-1} & o_{i-1} & c_{i-1} & v_{i-1} & p_{i-1} \\
	h_{i-2} & l_{i-2} & o_{i-2} & c_{i-2} & v_{i-2} & p_{i-2} \\
	\vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
	h_{i-x} & l_{i-x} & o_{i-x} & c_{i-x} & v_{i-x} & p_{i-x} \\
\end{bmatrix}
$$

The result of this process is an intuitively constructed training
example, consisting of an ordered time series of features over the $x$
previous days and a label indicating price movement in the near
future. Implementation of this process is discussed in the following
section.

We can take further steps to improve the quality of the resultant
training examples. Consider the case where the process described above
is applied to two adjacent records, $i$ and $i+1$. The result will be
two training examples with $x \times 6$ feature matrices which have a
substantial overlap. Specifically, we will produce feature matrices as
follows:

$$
\begin{aligned}
	features_i \in \mathbb{R}^{x \times 6} &=
	\begin{bmatrix}
		h_i & l_i & o_i & c_i & v_i & p_i \\
		\vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
		h_{i-x} & l_{i-x} & o_{i-x} & c_{i-x} & v_{i-x} & p_{i-x} \\
	\end{bmatrix}
	\\
	features_{i+1} \in \mathbb{R}^{x \times 6} &=
	\begin{bmatrix}
		h_{i+1} & l_{i+1} & o_{i+1} & c_{i+1} & v_{i+1} & p_{i+1} \\
		\vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
		h_{i-x+1} & l_{i-x+1} & o_{i-x+1} & c_{i-x+1} & v_{i-x+1} & p_{i-x+1} \\
	\end{bmatrix}
\end{aligned}
$$

Applying a stride greater than $1$ to the historical window will
reduce the amount of overlap between feature matrices, creating a
more diverse training set. For experimentation a stride of $5$ days
was chosen. This resulted in an adequate number of training examples
and minimized the overlap in the $3$ day future percent change
window.

Generating the feature matrices as described above is a
computationally expensive process, especially at small window
strides and large window sizes. To put this in perspective, consider a
training set produced from $1000$ stock symbols from $2008$ to $2018$
with a window size of $128$ days and a window stride of $1$. Assume
the stock only trades $200$ days out of the year.  The
number of resultant examples is given by

$$
\begin{aligned}
	N &= 1000 * 200 * (2018 - 2008) \\
	&= 2,000,000
\end{aligned}
$$

Similarly, if we consider this process applied the entirety of the
dataset without constraint

$$
\begin{aligned}
	N &= 8000 * 200 * (2018 - 1971) \\
	&= 75,200,000
\end{aligned}
$$

We will have a $128 \times 6$ feature matrix per row.
If we assume that each feature and the percent change is stored as a
$32$ bit float and the label is stored as an $8$ bit integer, we can
calculated the expected size of the resultant training set in

$$
\begin{aligned}
	features &= N \left(128 * 6 * 4\right) \\
	labels &= N \left(4 + 1\right) \\
	S &= labels + features \\
	&\approx 230 GB
\end{aligned}
$$